In [5]:
## library and datetime variable
from notebookutils import mssparkutils
import pandas as pd
from datetime import datetime
import pytz
from pyspark.sql.window import Window
from pyspark.sql.functions import *
from delta.tables import *
import json
# import pandas
# pd.set_option('display.max.columns', None)

autz = pytz.timezone('Australia/Melbourne')
dateformat = '%Y%m%d%H%M%S'
todaydatetime = datetime.now().astimezone(autz).strftime(dateformat)

StatementMeta(, 5ee512e7-a812-4c87-8edd-1c078edd66d6, 7, Finished, Available)

In [6]:
# # ## print variables from parent notebook
# watermarkcolumn = "columnname1"
# businesskey = "columnname2"
# lakehousename = "lakehousename"
# shortcutname = "shortcutfoldername"
# loadtype = "incremental"
# tablename = "tablename"

StatementMeta(, 5ee512e7-a812-4c87-8edd-1c078edd66d6, 8, Finished, Available)

In [7]:
## Create a archive folder in table directory
tabledirectory = 'Files/' + shortcutname + '/' + tablename
archivefolder = 'Files/' + shortcutname + '/' + tablename + '/archive/' 
numberarchivefiles = len([i for i in mssparkutils.fs.ls(tabledirectory) if 'archive' == i.name])
if numberarchivefiles == 0 : mssparkutils.fs.mkdirs(archivefolder)
else: pass
# print(archivefolder)

StatementMeta(, 5ee512e7-a812-4c87-8edd-1c078edd66d6, 9, Finished, Available)

In [24]:
## Get filename from directory
filepath = 'Files/' + shortcutname + '/' + tablename + '/'
filelist = [filepath + i.name for i in mssparkutils.fs.ls(filepath) if 'archive' not in i.name and '_delta_log' not in i.name ]
filelist.sort()
tablelist = [i.name for i in mssparkutils.fs.ls('Tables/') if tablename.lower() == i.name]

## Read file and save into table
## full load
if loadtype == 'full':
    if len(filelist)>0:

        # read and write file
        df = spark.read.parquet(filepath)
        df.write.format("delta").mode("overwrite").option("overwriteSchema","True").saveAsTable(tablename)
        numrows = df.count()

        ## move file into archive
        currpath = filelist[0]
        archivepath = filelist[0].replace(filepath, archivefolder).replace('.parquet', '_' + todaydatetime +'.parquet')
        mssparkutils.fs.mv(currpath, archivepath, overwrite=True)
        nb_result = 'full load - completed, overwrite'

    else: 
        numrows = 0
        nb_result = 'full load - completed, no new file'

# ## incremental load
elif loadtype == 'incremental':
    if len(tablelist)== 0 and len(filelist) > 0 :
        nb_result = 'incremental load - write file to a new table'

        ## read file and write into fabric table
        df = spark.read.parquet(filepath)
        df.write.format("delta").mode("overwrite").option("overwriteSchema","True").saveAsTable(tablename)
        numrows = df.count()

        ## move file into archive folder
        currpath = filelist[0]
        archivepath = filelist[0].replace(filepath, archivefolder)
        mssparkutils.fs.mv(currpath, archivepath, overwrite=True)
        print('from file - ' + filelist[0].replace(filepath,''))

    elif len(filelist) > 0 :
        nb_result = 'incremental load - upsert rows'
        for i in filelist:

            ## read fabric table, new file, then merge them
            tablenamelowercase = tablename.lower()
            dfold = DeltaTable.forPath(spark, "Tables/" + tablename.lower())
            dfnew = spark.read.format("parquet").load(filepath)
            (dfold.alias("o")\
                .merge(dfnew.alias("n"), f"o.{businesskey} == n.{businesskey}")\
                .whenMatchedUpdateAll()\
                .whenNotMatchedInsertAll()\
                .execute() )
            numrows = dfnew.count()

            ## move file into archive folder
            currpath = i
            archivepath = i.replace(filepath, archivefolder)
            mssparkutils.fs.mv(currpath, archivepath, overwrite=True)
            print('from file - ' + i.replace(filepath,''))
            
    else: 
        numrows = 0
        nb_result = 'incremental load - completed, no new file'

else: pass
print(nb_result)

StatementMeta(, 5ee512e7-a812-4c87-8edd-1c078edd66d6, 26, Finished, Available)

incremental load - upsert rows
from file - Filteredupbeat_transaction_daily_20231208010001.parquet


In [ ]:
## pass number of rows to the parent notebook
mssparkutils.notebook.exit(json.dumps({"rowswritten":numrows, "nbresult": nb_result}))